# Session 2

This session will provide an introduction to Cognitive Architectures (CA), a computational functional description and a theory of human-like minds.

Slides from the first lecture can be found [here](https://github.com/polina-tsvilodub/LMARL-course/tree/main/lmarl-course/slides/session2.pdf).

The discussion of the day will be on the paper by [Laird, Lebiere & Rosenbloom (2017). A Standard Model of the Mind: Toward a Common Computational Framework Across Artificial Intelligence, Cognitive Science, Neuroscience, and Robotics](https://ojs.aaai.org/aimagazine/index.php/aimagazine/article/view/2744).

## Exercises

Below are some questions to recap and apply central concepts of the second session. Please try answer them for yourself.
Further, exercises to familiarize yourself with theoretically-informed approaches of cognitive modeling.

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 2.1.: What are cognitive architectures?</span></strong>
>
> 1. What are the advantages and limitations of structured representations and a structured architecture? Name and briefly explain at least two advantages and disadvantages for each.
>
> 2. What is the conceptual difference between a cognitive architecture and the BDI model?
>
> 3. Consider the core modules of cognitive architectures: different types of memory, perception, motor modules, attention, action selection, learning. Which of these can one map onto aspects of LLMs? Map and explain briefly.
>
> 4. Compare the cognitive cycle described in the standard model and the architecture of Generative Agents. Which components match, which are missing or differ?
>
> 5. Do you consider the Generative Agents architecture a better simulator of human behavior that an LLM only? Name and briefly explain two arguments.
>
> 6. The Standard Model formalizes bounded rationality rather than fully optimal cognition. What would speak in favor, would would speak against having boundedly rational artificial agents, if they were a part of human every day life? Name one argument each.

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 1.2.: Aspects of cognitive architectures</span></strong>
>
> 1. One of the modules in cognitive architectures is perception, which interacts with memory. One core functionality of these subprocesses in human cogntion is categorization, i.e., assigning discrete categories to perceived stimuli, based on our knowledge (memory) of different categories. While ML models are highly accurate at categorization tasks, they need large amounts of training data that is implausible from a cognitive perspective. Please follow [this tutorial](https://alanjern.github.io/computational-psych-book/04-categorization.html) to see an example of a model from cognitive psychology where categorization happens based on a smaller number of examples.
>
>   a. What is the categorization "criterion" that is used?
>   b. What do you think are strengths and limitations of this model?
>
> 2. Selection of relevant aspects to remember is one of the most difficult aspects of building models that reliably interact with the complex real world. Re-using the code and LLM from session 1, compare different approaches to scoring the relevance of different observations of the environment. Your task is to compare these approaches to the relevance scoring used in the Generative Agents architecture (see prompt on slides). The different approaches are roughly described, but you need to formulate the respective prompts. Feel free to play around with the set of observations, too. Do you see differences depedning on scoring? Do the results conform your intuitions?

In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 44.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [2]:
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
# load the model
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [1]:
#### environment set up: ####
# set of observations
observations = [
    "It's sunny outside.",
    "You saw a big rock fall from the sky.",
    "You saw a tiny dog in front of your house.",
    "You met your best friend today.",
    "You met your partner whom you haven't seen for a long time.",
    "You found your high school diploma.",
    "You ate cereal for lunch."
]
# current memory
memory = [
    "You have a college diploma.",
    "You live in a small town.",
    "You have a cat."
]

In [ ]:
# Approach 1: relevance of remembering is based on similarity of current memories and the observation

In [ ]:
# Approach 2: relevance is based on importance of the observation for generalization, success in the future --
# e.g., could roughly approximated by expected number of situations which might rely on this knowledge

In [ ]:
# Approach 3: relevance scoring prompting from Generative agents

In [ ]:
# Approach 4 (more advanced): multi-step prompting: rate relevance following generative agents, and also unexpectedness of the observation
# in two separate LM calls and then rate overall retention score based on the predicted scores